# <center> **NLP analysis of jokes dataset**

## Table of Contents

1. **[Functions](#1)**
2. **[Preprocessing](#3)**
	* [Cleaning](#sub-heading2)

# **Imports**

In [1]:
!pip install unidecode
!pip install emoji
!pip install word2number
!pip install contractions
!pip3 install spacy
!python3 -m spacy download en_core_web_sm
!python3 -m nltk.downloader stopwords
!pip install pyspellchecker
!pip install iplot

Defaulting to user installation because normal site-packages is not writeable


You should consider upgrading via the 'C:\Users\anaconda3\python.exe -m pip install --upgrade pip' command.


Defaulting to user installation because normal site-packages is not writeable


You should consider upgrading via the 'C:\Users\anaconda3\python.exe -m pip install --upgrade pip' command.


Defaulting to user installation because normal site-packages is not writeable


...

"python3" ­Ґ пў«пҐвбп ў­гваҐ­­Ґ© Ё«Ё ў­Ґи­Ґ©
Є®¬ ­¤®©, ЁбЇ®«­пҐ¬®© Їа®Ја ¬¬®© Ё«Ё Ї ЄҐв­л¬ д ©«®¬.
"python3" ­Ґ пў«пҐвбп ў­гваҐ­­Ґ© Ё«Ё ў­Ґи­Ґ©
Є®¬ ­¤®©, ЁбЇ®«­пҐ¬®© Їа®Ја ¬¬®© Ё«Ё Ї ЄҐв­л¬ д ©«®¬.


Defaulting to user installation because normal site-packages is not writeable


You should consider upgrading via the 'C:\Users\anaconda3\python.exe -m pip install --upgrade pip' command.


Defaulting to user installation because normal site-packages is not writeable


You should consider upgrading via the 'C:\Users\anaconda3\python.exe -m pip install --upgrade pip' command.


In [2]:
!pip install gensim

Defaulting to user installation because normal site-packages is not writeable


You should consider upgrading via the 'C:\Users\anaconda3\python.exe -m pip install --upgrade pip' command.


In [3]:
import pandas as pd
import numpy as np
import re
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import style
style.use('seaborn')
%matplotlib inline
#graphs in svg look clearer
%config InlineBackend.figure_format = 'svg' 
import warnings
warnings.simplefilter('ignore')

from scipy.stats import norm

# from google.colab import drive
import os
import string
import emoji

import plotly as py
import numpy as np
from plotly.offline import init_notebook_mode, iplot
from plotly.graph_objs import Contours, Histogram2dContour, Marker, Scatter

import unidecode
from word2number import w2n
import contractions

import spacy

import nltk
nltk.download('punkt')
nltk.download('wordnet')
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
nltk.download('averaged_perceptron_tagger')
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from collections import Counter
from spellchecker import SpellChecker
# from wordcloud import WordCloud

from tqdm._tqdm_notebook import tqdm_notebook
tqdm_notebook.pandas(desc='PROGRESS>>>')

# from google_drive_downloader import GoogleDriveDownloader as gdd
# from IPython.core.display import display, HTML

from sklearn.model_selection import RepeatedStratifiedKFold, GridSearchCV, StratifiedKFold, KFold, cross_val_score, train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import metrics
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import label_binarize, StandardScaler

from torch.optim.lr_scheduler import CosineAnnealingLR
from tqdm import tqdm
# import gensim
# import gensim.downloader as api
# from gensim.models import Word2Vec
# import gensim.downloader as gensim_api




[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Полина\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Полина\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Полина\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [4]:
!pip install torchvision

Defaulting to user installation because normal site-packages is not writeable


You should consider upgrading via the 'C:\Users\anaconda3\python.exe -m pip install --upgrade pip' command.


In [5]:
from torch.nn.utils.rnn import pad_sequence
from functools import partial
import torch      
import torch.nn as nn
from torch.utils.data import TensorDataset, ConcatDataset, DataLoader, random_split, Dataset
import torch.nn.functional as F
import torchvision.transforms as transforms
import torchvision
import torch.optim as optim

In [6]:
import cufflinks as cf
cf.go_offline()
cf.set_config_file(offline=False, world_readable=True)
import plotly.graph_objs as go

In [7]:
!pip install bs4 

Defaulting to user installation because normal site-packages is not writeable


You should consider upgrading via the 'C:\Users\anaconda3\python.exe -m pip install --upgrade pip' command.


In [8]:
# !pip3 install beautifulsoup4

In [9]:
from bs4 import BeautifulSoup

In [10]:
data = pd.read_csv(r'C:\NLP_humor\data\jokes_dataset.csv', 
                 sep=',', 
                 encoding='utf-8').set_index('id')
# work with copy
df = data.copy(deep=True)

# **Functions**

In [11]:
def draw_histogram(data):
    f,ax_hist = plt.subplots(1)
    sns.distplot(data, hist=True,  kde=True, rug=True, bins=10, fit=norm, ax=ax_hist);
    plt.xlabel("values")
    plt.title("Distribution") 

    mean=data.mean()

    ax_hist.axvline(mean, color='r', linestyle='--', label=f"Mean={mean:.3f}")
    ax_hist.legend()

    plt.show()

**Removal of html tags**

In [12]:
def strip_html_tags(text):
    """remove html tags from text"""
    soup = BeautifulSoup(text, "html.parser")
    stripped_text = soup.get_text(separator=" ")
    return stripped_text

**Removal of whitespaces**

In [13]:
def remove_whitespace(text):
    """remove extra whitespaces from text"""
    text = text.strip()
    return " ".join(text.split())

**Removal of accented characters**

In [14]:
def remove_accented_chars(text):
    """remove accented characters from text, e.g. café"""
    text = unidecode.unidecode(text)
    return text

**Removal of shortened words**

In [15]:
def expand_contractions(text):
    """expand shortened words, e.g. don't to do not"""
    text = contractions.fix(text)
    return text

**Removal of urls**

In [16]:
def find_urls(text):
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.findall(text)

In [17]:
def remove_urls(text):
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', text)

**Removal of Frequent words**

In [18]:
def count_words(text, top=10):
    cnt = Counter()
    for text in text.values:
        for word in text.split():
            cnt[word] += 1
          
    return cnt.most_common(top)

In [19]:
# FREQWORDS = set([w for (w, wc) in count_words(data['joke'], 10)])
# FREQWORDS

# n_rare_words = 10
# RAREWORDS = set([w for (w, wc) in count_words(data['joke'][:-n_rare_words-1:-1])])
# RAREWORDS

In [20]:
def remove_freqwords(text):
    """remove the frequent words"""
    return " ".join([word for word in str(text).split() if word not in FREQWORDS])

**Removal of Rare words**

In [21]:
def remove_rarewords(text):
    """remove the rare words"""
    return " ".join([word for word in str(text).split() if word not in RAREWORDS])

**Removal of stopwords**


In [22]:
STOPWORDS = set(stopwords.words('english'))

def remove_stopwords(text):
    """remove the stopwords"""
    return " ".join([word for word in str(text).split() if word not in STOPWORDS])

**Convert chat words**

In [23]:
chat_words_str = """
AFAIK=As Far As I Know
AFK=Away From Keyboard
ASAP=As Soon As Possible
ATK=At The Keyboard
ATM=At The Moment
A3=Anytime, Anywhere, Anyplace
BAK=Back At Keyboard
BBL=Be Back Later
BBS=Be Back Soon
BFN=Bye For Now
B4N=Bye For Now
BRB=Be Right Back
BRT=Be Right There
BTW=By The Way
B4=Before
B4N=Bye For Now
CU=See You
CUL8R=See You Later
CYA=See You
FAQ=Frequently Asked Questions
FC=Fingers Crossed
FWIW=For What It's Worth
FYI=For Your Information
GAL=Get A Life
GG=Good Game
GN=Good Night
GMTA=Great Minds Think Alike
GR8=Great!
G9=Genius
IC=I See
ICQ=I Seek you (also a chat program)
ILU=ILU: I Love You
IMHO=In My Honest/Humble Opinion
IMO=In My Opinion
IOW=In Other Words
IRL=In Real Life
LDR=Long Distance Relationship
LMAO=Laugh My A.. Off
LOL=Laughing Out Loud
LTNS=Long Time No See
L8R=Later
MTE=My Thoughts Exactly
M8=Mate
NRN=No Reply Necessary
OIC=Oh I See
PITA=Pain In The A..
PRT=Party
PRW=Parents Are Watching
ROFL=Rolling On The Floor Laughing
ROFLOL=Rolling On The Floor Laughing Out Loud
ROTFLMAO=Rolling On The Floor Laughing My A.. Off
SK8=Skate
STATS=Your sex and age
ASL=Age, Sex, Location
THX=Thank You
TTFN=Ta-Ta For Now!
TTYL=Talk To You Later
U=You
U2=You Too
U4E=Yours For Ever
WB=Welcome Back
WTF=What The F...
WTG=Way To Go!
WUF=Where Are You From?
W8=Wait...
7K=Sick:-D Laugher
"""

In [24]:
def chat_words_conversion(text):
    new_text = []
    for w in text.split():
        if w.upper() in chat_words_list:
            new_text.append(chat_words_map_dict[w.upper()])
        else:
            new_text.append(w)
    return " ".join(new_text)

**Convertion of emoticons**

In [25]:
EMOTICONS = {
    u":‑\)":"Happy face or smiley",
    u":\)":"Happy face or smiley",
    u":-\]":"Happy face or smiley",
    u":\]":"Happy face or smiley",
    u":-3":"Happy face smiley",
    u":3":"Happy face smiley",
    u":->":"Happy face smiley",
    u":>":"Happy face smiley",
    u"8-\)":"Happy face smiley",
    u":o\)":"Happy face smiley",
    u":-\}":"Happy face smiley",
    u":\}":"Happy face smiley",
    u":-\)":"Happy face smiley",
    u":c\)":"Happy face smiley",
    u":\^\)":"Happy face smiley",
    u"=\]":"Happy face smiley",
    u"=\)":"Happy face smiley",
    u":‑D":"Laughing, big grin or laugh with glasses",
    u":D":"Laughing, big grin or laugh with glasses",
    u"8‑D":"Laughing, big grin or laugh with glasses",
    u"8D":"Laughing, big grin or laugh with glasses",
    u"X‑D":"Laughing, big grin or laugh with glasses",
    u"XD":"Laughing, big grin or laugh with glasses",
    u"=D":"Laughing, big grin or laugh with glasses",
    u"=3":"Laughing, big grin or laugh with glasses",
    u"B\^D":"Laughing, big grin or laugh with glasses",
    u":-\)\)":"Very happy",
    u":‑\(":"Frown, sad, andry or pouting",
    u":-\(":"Frown, sad, andry or pouting",
    u":\(":"Frown, sad, andry or pouting",
    u":‑c":"Frown, sad, andry or pouting",
    u":c":"Frown, sad, andry or pouting",
    u":‑<":"Frown, sad, andry or pouting",
    u":<":"Frown, sad, andry or pouting",
    u":‑\[":"Frown, sad, andry or pouting",
    u":\[":"Frown, sad, andry or pouting",
    u":-\|\|":"Frown, sad, andry or pouting",
    u">:\[":"Frown, sad, andry or pouting",
    u":\{":"Frown, sad, andry or pouting",
    u":@":"Frown, sad, andry or pouting",
    u">:\(":"Frown, sad, andry or pouting",
    u":'‑\(":"Crying",
    u":'\(":"Crying",
    u":'‑\)":"Tears of happiness",
    u":'\)":"Tears of happiness",
    u"D‑':":"Horror",
    u"D:<":"Disgust",
    u"D:":"Sadness",
    u"D8":"Great dismay",
    u"D;":"Great dismay",
    u"D=":"Great dismay",
    u"DX":"Great dismay",
    u":‑O":"Surprise",
    u":O":"Surprise",
    u":‑o":"Surprise",
    u":o":"Surprise",
    u":-0":"Shock",
    u"8‑0":"Yawn",
    u">:O":"Yawn",
    u":-\*":"Kiss",
    u":\*":"Kiss",
    u":X":"Kiss",
    u";‑\)":"Wink or smirk",
    u";\)":"Wink or smirk",
    u"\*-\)":"Wink or smirk",
    u"\*\)":"Wink or smirk",
    u";‑\]":"Wink or smirk",
    u";\]":"Wink or smirk",
    u";\^\)":"Wink or smirk",
    u":‑,":"Wink or smirk",
    u";D":"Wink or smirk",
    u":‑P":"Tongue sticking out, cheeky, playful or blowing a raspberry",
    u":P":"Tongue sticking out, cheeky, playful or blowing a raspberry",
    u"X‑P":"Tongue sticking out, cheeky, playful or blowing a raspberry",
    u"XP":"Tongue sticking out, cheeky, playful or blowing a raspberry",
    u":‑Þ":"Tongue sticking out, cheeky, playful or blowing a raspberry",
    u":Þ":"Tongue sticking out, cheeky, playful or blowing a raspberry",
    u":b":"Tongue sticking out, cheeky, playful or blowing a raspberry",
    u"d:":"Tongue sticking out, cheeky, playful or blowing a raspberry",
    u"=p":"Tongue sticking out, cheeky, playful or blowing a raspberry",
    u">:P":"Tongue sticking out, cheeky, playful or blowing a raspberry",
    u":‑/":"Skeptical, annoyed, undecided, uneasy or hesitant",
    u":/":"Skeptical, annoyed, undecided, uneasy or hesitant",
    u":-[.]":"Skeptical, annoyed, undecided, uneasy or hesitant",
    u">:[(\\\)]":"Skeptical, annoyed, undecided, uneasy or hesitant",
    u">:/":"Skeptical, annoyed, undecided, uneasy or hesitant",
    u":[(\\\)]":"Skeptical, annoyed, undecided, uneasy or hesitant",
    u"=/":"Skeptical, annoyed, undecided, uneasy or hesitant",
    u"=[(\\\)]":"Skeptical, annoyed, undecided, uneasy or hesitant",
    u":L":"Skeptical, annoyed, undecided, uneasy or hesitant",
    u"=L":"Skeptical, annoyed, undecided, uneasy or hesitant",
    u":S":"Skeptical, annoyed, undecided, uneasy or hesitant",
    u":‑\|":"Straight face",
    u":\|":"Straight face",
    u":$":"Embarrassed or blushing",
    u":‑x":"Sealed lips or wearing braces or tongue-tied",
    u":x":"Sealed lips or wearing braces or tongue-tied",
    u":‑#":"Sealed lips or wearing braces or tongue-tied",
    u":#":"Sealed lips or wearing braces or tongue-tied",
    u":‑&":"Sealed lips or wearing braces or tongue-tied",
    u":&":"Sealed lips or wearing braces or tongue-tied",
    u"O:‑\)":"Angel, saint or innocent",
    u"O:\)":"Angel, saint or innocent",
    u"0:‑3":"Angel, saint or innocent",
    u"0:3":"Angel, saint or innocent",
    u"0:‑\)":"Angel, saint or innocent",
    u"0:\)":"Angel, saint or innocent",
    u":‑b":"Tongue sticking out, cheeky, playful or blowing a raspberry",
    u"0;\^\)":"Angel, saint or innocent",
    u">:‑\)":"Evil or devilish",
    u">:\)":"Evil or devilish",
    u"\}:‑\)":"Evil or devilish",
    u"\}:\)":"Evil or devilish",
    u"3:‑\)":"Evil or devilish",
    u"3:\)":"Evil or devilish",
    u">;\)":"Evil or devilish",
    u"\|;‑\)":"Cool",
    u"\|‑O":"Bored",
    u":‑J":"Tongue-in-cheek",
    u"#‑\)":"Party all night",
    u"%‑\)":"Drunk or confused",
    u"%\)":"Drunk or confused",
    u":-###..":"Being sick",
    u":###..":"Being sick",
    u"<:‑\|":"Dump",
    u"\(>_<\)":"Troubled",
    u"\(>_<\)>":"Troubled",
    u"\(';'\)":"Baby",
    u"\(\^\^>``":"Nervous or Embarrassed or Troubled or Shy or Sweat drop",
    u"\(\^_\^;\)":"Nervous or Embarrassed or Troubled or Shy or Sweat drop",
    u"\(-_-;\)":"Nervous or Embarrassed or Troubled or Shy or Sweat drop",
    u"\(~_~;\) \(・\.・;\)":"Nervous or Embarrassed or Troubled or Shy or Sweat drop",
    u"\(-_-\)zzz":"Sleeping",
    u"\(\^_-\)":"Wink",
    u"\(\(\+_\+\)\)":"Confused",
    u"\(\+o\+\)":"Confused",
    u"\(o\|o\)":"Ultraman",
    u"\^_\^":"Joyful",
    u"\(\^_\^\)/":"Joyful",
    u"\(\^O\^\)／":"Joyful",
    u"\(\^o\^\)／":"Joyful",
    u"\(__\)":"Kowtow as a sign of respect, or dogeza for apology",
    u"_\(\._\.\)_":"Kowtow as a sign of respect, or dogeza for apology",
    u"<\(_ _\)>":"Kowtow as a sign of respect, or dogeza for apology",
    u"<m\(__\)m>":"Kowtow as a sign of respect, or dogeza for apology",
    u"m\(__\)m":"Kowtow as a sign of respect, or dogeza for apology",
    u"m\(_ _\)m":"Kowtow as a sign of respect, or dogeza for apology",
    u"\('_'\)":"Sad or Crying",
    u"\(/_;\)":"Sad or Crying",
    u"\(T_T\) \(;_;\)":"Sad or Crying",
    u"\(;_;":"Sad of Crying",
    u"\(;_:\)":"Sad or Crying",
    u"\(;O;\)":"Sad or Crying",
    u"\(:_;\)":"Sad or Crying",
    u"\(ToT\)":"Sad or Crying",
    u";_;":"Sad or Crying",
    u";-;":"Sad or Crying",
    u";n;":"Sad or Crying",
    u";;":"Sad or Crying",
    u"Q\.Q":"Sad or Crying",
    u"T\.T":"Sad or Crying",
    u"QQ":"Sad or Crying",
    u"Q_Q":"Sad or Crying",
    u"\(-\.-\)":"Shame",
    u"\(-_-\)":"Shame",
    u"\(一一\)":"Shame",
    u"\(；一_一\)":"Shame",
    u"\(=_=\)":"Tired",
    u"\(=\^\·\^=\)":"cat",
    u"\(=\^\·\·\^=\)":"cat",
    u"=_\^=	":"cat",
    u"\(\.\.\)":"Looking down",
    u"\(\._\.\)":"Looking down",
    u"\^m\^":"Giggling with hand covering mouth",
    u"\(\・\・?":"Confusion",
    u"\(?_?\)":"Confusion",
    u">\^_\^<":"Normal Laugh",
    u"<\^!\^>":"Normal Laugh",
    u"\^/\^":"Normal Laugh",
    u"\（\*\^_\^\*）" :"Normal Laugh",
    u"\(\^<\^\) \(\^\.\^\)":"Normal Laugh",
    u"\(^\^\)":"Normal Laugh",
    u"\(\^\.\^\)":"Normal Laugh",
    u"\(\^_\^\.\)":"Normal Laugh",
    u"\(\^_\^\)":"Normal Laugh",
    u"\(\^\^\)":"Normal Laugh",
    u"\(\^J\^\)":"Normal Laugh",
    u"\(\*\^\.\^\*\)":"Normal Laugh",
    u"\(\^—\^\）":"Normal Laugh",
    u"\(#\^\.\^#\)":"Normal Laugh",
    u"\（\^—\^\）":"Waving",
    u"\(;_;\)/~~~":"Waving",
    u"\(\^\.\^\)/~~~":"Waving",
    u"\(-_-\)/~~~ \($\·\·\)/~~~":"Waving",
    u"\(T_T\)/~~~":"Waving",
    u"\(ToT\)/~~~":"Waving",
    u"\(\*\^0\^\*\)":"Excited",
    u"\(\*_\*\)":"Amazed",
    u"\(\*_\*;":"Amazed",
    u"\(\+_\+\) \(@_@\)":"Amazed",
    u"\(\*\^\^\)v":"Laughing,Cheerful",
    u"\(\^_\^\)v":"Laughing,Cheerful",
    u"\(\(d[-_-]b\)\)":"Headphones,Listening to music",
    u'\(-"-\)':"Worried",
    u"\(ーー;\)":"Worried",
    u"\(\^0_0\^\)":"Eyeglasses",
    u"\(\＾ｖ\＾\)":"Happy",
    u"\(\＾ｕ\＾\)":"Happy",
    u"\(\^\)o\(\^\)":"Happy",
    u"\(\^O\^\)":"Happy",
    u"\(\^o\^\)":"Happy",
    u"\)\^o\^\(":"Happy",
    u":O o_O":"Surprised",
    u"o_0":"Surprised",
    u"o\.O":"Surpised",
    u"\(o\.o\)":"Surprised",
    u"oO":"Surprised",
    u"\(\*￣m￣\)":"Dissatisfied",
    u"\(‘A`\)":"Snubbed or Deflated"
}

In [26]:
def convert_emoticons(text):
    for emot in EMOTICONS:
        text = re.sub(u'('+emot+')', "_".join(EMOTICONS[emot].replace(",","").split()), text)
    return text

**Removal of Punctuations**

In [27]:
PUNCT_TO_REMOVE = string.punctuation

def remove_punctuation(text):
    """custom function to remove the punctuation"""
    return text.translate(str.maketrans('', '', PUNCT_TO_REMOVE))

**Removal of numbers**

In [28]:
def remove_numbers(input):
    input_str = re.sub(r'\d+', "", input)
    return input_str

**Find emoji**

In [29]:
def extract_emojis(s):
    return ''.join(c for c in s if c in emoji.UNICODE_EMOJI['en'])

**Stemming**

In [30]:
stemmer = PorterStemmer()

def stem_words(text):
    return " ".join([stemmer.stem(word) for word in text])

**Word importance**

In [31]:
# def get_word_importance(model,tfidf, top=15):

#     important_tokens = pd.DataFrame(
#         data=model.coef_[0],
#         index=tfidf.get_feature_names(),
#         columns=['Coefs']
#     )

#     important_tokens_pos = important_tokens.sort_values(by='Coefs', ascending=False)[:top]
#     important_tokens_neg = important_tokens.sort_values(by='Coefs', ascending=False)[-top:]

#     word_imp = important_tokens_pos.copy(deep=True)
#     word_imp = word_imp.append(important_tokens_neg)

#     plt.figure(figsize=(15,10))
#     sns.barplot(x=word_imp.index, y='Coefs', data=word_imp)
#     plt.title('Word importance')
#     plt.xlabel('POS             vs.               NEG')
#     plt.ylabel('word weight')
#     plt.xticks(rotation=55)
#     plt.show()

# **Preprocessing**

In [32]:
df.shape

(194616, 3)

In [33]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 194616 entries, 5tz52q to 1a801u
Data columns (total 3 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   body    190507 non-null  object 
 1   score   194553 non-null  float64
 2   title   194553 non-null  object 
dtypes: float64(1), object(2)
memory usage: 5.9+ MB


## **Check missing data**

In [34]:
df.isnull().sum()

body     4109
score      63
title      63
dtype: int64

In [35]:
df[df['body'].isna() == True]

,body,score,title
id,,,
5tr0co,NaN,4.0,Obese people are a very large portion of the p...
4zcsif,NaN,2.0,"A priest, a bishop, and a pontiff are all in a..."
4toft6,NaN,0.0,Plagiarism is cheating. If mrs trump cheated o...
4rlhug,NaN,0.0,Marriage is like childhood...
4rlf8f,NaN,0.0,The worst excuse I have ever came up with.
...,...,...,...
1avalc,NaN,0.0,a dyslexic man walks into a bra
1autf7,NaN,22.0,I tried searching on Google for 'Lost Medieval...
1ajoog,NaN,0.0,591


There are not a lot of NaNs in 2 columns (score, title), we can just drop them.

In [36]:
df.dropna(subset=['title', 'score'], inplace=True)

So, now we have full jokes in body, not in title.

In [37]:
df.isna().sum()

body     4109
score       0
title       0
dtype: int64

Now, we need to work with jokes without titles --> there I replace NaNs with " ".

In [38]:
df['body'] = df['body'].fillna('')

In [39]:
df.isna().sum()

body     0
score    0
title    0
dtype: int64

In [40]:
df['score'] = df['score'].astype(np.int32)

### **Make new df with full joke text**

In [41]:
df["joke"] = df["title"] + " " + df["body"]

In [42]:
df.drop(['title', 'body'], axis=1, inplace=True)

In [43]:
df.head()

,score,joke
id,,
5tz52q,1,I hate how you cant even say black paint anymo...
5tz4dd,0,What's the difference between a Jew in Nazi Ge...
5tz319,0,I recently went to America.... ...and being th...
5tz2wj,1,"Brian raises his hand and says, “He’s in Heave..."
5tz1pc,0,You hear about the University book store worke...


### **Dropping duplicates**

In [44]:
df.shape

(194553, 2)

In [45]:
df.drop_duplicates(inplace=True)
df.shape

(193906, 2)

## **Cleaning**

In [46]:
text_col = 'joke'

**Lower Casing**  
Lower casing is a common text preprocessing technique. The idea is to convert the input text into same casing format so that 'text', 'Text' and 'TEXT' are treated the same way.  

This is more helpful for text featurization techniques like frequency, tfidf as it helps to combine the same words together thereby reducing the duplication and get correct counts / tfidf values.  

**Some other parts like:** *html removal, extra whitespaces removal are also important.*

These may not be helpful when we do tasks like Part of Speech tagging (where proper casing gives some information about Nouns and so on) and Sentiment Analysis (where upper casing refers to anger and so on)  

In [47]:
def text_preprocessing(df, col):
    
    #remove html tags
    df[col] = df[col].apply(strip_html_tags)
    print('>>> REMOVE HTML TAGS <<<')
    print(df.head(1), '\n')

    #remove extra whitespaces
    df[col] = df[col].apply(remove_whitespace)
    print('>>> REMOVE EXTRA WHITESPACES <<<')
    print(data.head(1), '\n')

    #remove accented characters
    df[col] =  df[col].apply(remove_accented_chars)
    print('>>> REMOVE ACCENTED CHARS <<<')
    print(df.head(1), '\n')

    #expand contractions
    df[col] =  df[col].apply(expand_contractions)
    print('>>> EXPAND CONTRACTIONS <<<')
    print(data.head(1), '\n')


In [48]:
text_preprocessing(df, col='joke')

>>> REMOVE HTML TAGS <<<
        score                                               joke
id                                                              
5tz52q      1  I hate how you cant even say black paint anymo... 

>>> REMOVE EXTRA WHITESPACES <<<
                                                     body  score  \
id                                                                 
5tz52q  Now I have to say "Leroy can you please paint ...    1.0   

                                                   title  
id                                                        
5tz52q  I hate how you cant even say black paint anymore   

>>> REMOVE ACCENTED CHARS <<<
        score                                               joke
id                                                              
5tz52q      1  I hate how you cant even say black paint anymo... 

>>> EXPAND CONTRACTIONS <<<
                                                     body  score  \
id                                    

**Removal of URLs**  
Next preprocessing step is to remove any URLs present in the data. Probably we might need to remove them for our further analysis.


1st of all, I check whether there are any URLs in dataset:

In [49]:
for i in df['joke']:
    url = find_urls(i)
    if url == []:
        continue
    else:
        print(url)
      

['www.....That']
['http://www.youtube.com/watch?v=xaFZrxlPwWs),']
['http://m.imgur.com/x29gyvN', 'http://m.imgur.com/Q1iyUoT', 'http://m.imgur.com/hKRnGrT)', 'http://m.imgur.com/fqufGbK)']
['http://www.afterfeed.com/)?"']
['https://www.reddit.com/r/Punny/comments/4zob6u/tonight_the_world_egg_throwing_federation_host_a/?st=ISBQLINS&sh=8da1321d']
['www..."']
['http://www.merriam-webster.com/dictionary/hypocorism)."']
['https://techcrunch.com/2016/08/11/reddit-is-currently-experiencing-a-major-outage/']
['https://m.vk.com/wall-55955185_3834']
['https://en.m.wikipedia.org/wiki/Umami']
['http://www.afterfeed.com/story/detail/13350/14-epic-jokes-by-chandler-bing-from-friends-that-will-make-hole-in-your-belly)']
['http://www.dictionary.com/browse/evacuate)']
['http://instantrimshot.com/)']
['http://www.dictionary.com/browse/elope?s=t)']
['http://you-tricks.blogspot.com/2016/07/asian-doctor.html)']
['www..']
['https://pbs.twimg.com/media/CjVBbALUoAAb6r_.jpg)']
['www.curing-conjunctivitis.com']

['http://www.jokeindex.com/joke.asp?Joke=3584']
['http://thoughtcatalog.com/derek-marshall/2013/11/20-seriously-dark-anthony-jeselnik-jokes-that']
['www.JebBush.com']
['http://imgur.com/yzelyVT)']
['http://blog.counter-strike.net/index.php/2015/12/13288/']
['https://search.yahoo.com/yhs/search?p=i+can%27t+feel+my+face&ei=UTF-8&hspart=mozilla&hsimp=yhs-004']
['https://www.reddit.com/r/Jokes/comments/3vm6mw)']
['http://www.chemicalelements.com/elements/uuu.html)']
['http://makeagif.com/vY9iUJ)']
['http://www.mlive.com/wolverines/index.ssf/2015/12/michigans_jake_butt_wins_big_t.html']
['https://upload.wikimedia.org/wikipedia/commons/3/39/Monument_Valley_Merrick_Butte.jpg)']
['https://i.imgur.com/gKNf5ch.jpg)']
['https://np.reddit.com/r/AskReddit/comments/3v4b1d/what_didnt_you_notice_about_yourself_until/cxki21e']
['https://en.wikipedia.org/wiki/National_Sorry_Day']
['http://hugeforanasian.com/product/huge-for-an-asian/']
['https://www.reddit.com/r/AskReddit/comments/3unb5b/police_officers

['http://funshayari.com/aashiq-banaya-aapne/']
['http://www.funny-jokes-quotes.com/daily-life-situations.html#UQbK0SvD6QcItByk.99']
['https://www.reddit.com/r/AskReddit/comments/3jul1x/what_critically_aclaimed_videogame_did_you_hate/cuslgfe']
['http://imgur.com/YRGs7P0)']
['http://imgur.com/qe7LBx5']
['http://i.imgur.com/cx13xBc.png']
['http://www.ahajokes.com/age49.html)']
['https://en.wikipedia.org/wiki/Bhikkhuni)']
['http://zelda.wikia.com/wiki/Link)']
['http://www.fec.gov/press/resources/2016presidential_form2nm.shtml)']
['https://www.reddit.com/r/Jokes/comments/3jm30p/why_business_people_continue_to_earn_more_than/']
['http://i.imgur.com/aNurRdN.jpg](http://i.imgur.com/aNurRdN.jpg)']
['http://www.davidpbrown.co.uk/jokes/european-commission.html)']
['http://i.imgur.com/b9IbEUv.jpgs']
['https://www.reddit.com/r/AskReddit/comments/yykyd/can_anyone_come_up_with_the_end_of_the_a_naked/c5zymvw']
['http://chronicle.com/blogs/linguafranca/2015/09/02/best-linguistic-jokes-of-the-2015-fring

['http://www.sickipedia.org/racism/french/my-new-housemate-is-french-last-night-when-he-got-1644723']
['http://www.usa.gov/directory/federal/)']
['https://en.wikipedia.org/wiki/Dee_Barnes).']
['http://cdn.charismanews.com/images/stories/2015/featured-news/Rachel-Dolezal.jpg)']
['https://scontent-mia1-1.xx.fbcdn.net/hphotos-xap1/v/t1.0-9/s720x720/11202960_858688867539955_9213673678453179536_n.jpg?oh=c29b4a874228ceae54190d54b5244993&oe=562CFB25)']
['http://www.google.com/search?hl=en&q=recursion', 'http://clickfifty.com/11-super-nerd-jokes-that']
['http://www.reddit.com/r/pussypassdenied/comments/39z116/rachel_dolezal_resigns_as_president_of_naacp/cs94d83']
['http://www.reddit.com/r/Jokes/comments/2q7fb1/multilevel_meta_joke/)']
['http://www.reddit.com/r/Jokes/comments/39zo5p/wanna_hear_a_joke_about_recursion/)']
['http://i.imgur.com/DbgAFxp.jpg)']
['http://www.imdb.com/character/ch0289864/)', 'http://www.imdb.com/character/ch0371100/)']
['https://twitter.com/AkposTheComedia)']
['http://

['http://imgur.com/r/dasa2828/r73VJe3']
['http://i.imgur.com/9ohNxGM.jpg)']
['http://i.imgur.com/VUmou.gif)']
['http://www.babycenter.com/baby-names-beronica-233035.htm#ccPostCommentContainerAnchor']
['http://cdn.webfail.com/upl/img/076d053aa99/post2.jpg)']
['http://imgur.com/a/XKkQB']
['https://en.wikipedia.org/wiki/Mohs_scale_of_mineral_hardness#Minerals).']
['http://www.reddit.com/r/Jokes/comments/2yhlf0/a_man_goes_to_a_bar/)']
['http://imgur.com/OJKPTJr']
['http://en.m.wikipedia.org/wiki/Vine_%28service%29)']
['https://www.comedy.co.uk/guide/tv/qi/episodes/6/10/)']
['http://phys.org/news/2015-03-giant-methane-storms-uranus.html']
['http://www.reddit.com/r/science/comments/2ybgx8/giant_methane_storms_on_uranus/)']
['http://www.phespirit.info/montypython/four_yorkshiremen.htm']
['http://www.jokecenter.com/jokes/Sex/650.htm)']
['http://i.imgur.com/giiRBOi.jpghttp://i.imgur.com/1DYBcoH.jpg']
['http://www.telegraph.co.uk/technology/apple/8841347/Steve-Jobs-regretted-trying-to-beat-cance

['http://www.wolframalpha.com/input/?i=What%27s+one+cracker+plus+two+crackers%3F)']
['http://www.badgolfer.com/departments/jokes/woman-golfers1.htm)']
['http://travelmediacafe.com/2014/11/18/10-money-saving-tips/']
['http://www.redditstatic.com/reddit500.png']
['http://jokes.cc.com/funny-walks-into-a-bar']
['http://www.yoaca.com/jokes/the-smart-parrot/)']
['http://www.thalidomide.ca/recognition-of-thalidomide-defects/)']
['http://en.wikipedia.org/wiki/Barbiturate),']
['http://www.yoaca.com/health-medical/inside-thoughts-of-a-doctor/)']
['http://www.reddit.com/r/AskReddit/comments/2j7aff/z/cl9844o']
['http://www.snopes.com/language/literary/babyshoes.asp)']
['http://www.reddit.com/r/videos/comments/2lvhr1/girl_gets_caught_cheating_by_best_man_story_in/clym8e6).']
['http://smile.xonaki.com/Joke/EN?categoryCode=EN&jokeId=37)']
['http://www.reddit.com/r/worldnews/comments/2lub07/isis_selling_oil_as_low_as_20_per_barrel/clyd5lj']
['http://smile.xonaki.com/Joke/EN?categoryCode=EN&jokeId=34#m

['http://i.imgur.com/6SvvADP.jpg)']
['http://youtu.be/lMBVMyXaCng)']
['http://www.amazon.co.uk/Zizeks-Jokes-About-Hegel-Negation/dp/0262026716)']
['http://www.reddit.com/r/pics/comments/2ee16p/a_friend_was_at_a_bar_a_guy_walked_in_paid_for_a/)']
['http://www.reddit.com/r/AskHistorians/comments/2ec7av/what_was_comedy_like_in_nazi_germany/cjy52g4](http://www.reddit.com/r/AskHistorians/comments/2ec7av/what_was_comedy_like_in_nazi_germany/cjy52g4)']
['https://www.youtube.com/watch?v=GQjM5qsVryw&list=LLbBkuEQNU6MBnFuJnbb_p0Q&index=5)']
['http://imgur.com/65aBahK)']
['http://www.reddit.com/r/Jokes/comments/2e9hgg/american_tourists_visit_russia/cjxf7p4)']
['http://www.space.com/26888-sea-plankton-space-station-russian-claim.html)']
['https://www.youtube.com/watch?v=TzwDgz053PI)']
['http://memecrunch.com/meme/1MG0Q/big-summer-blowout/image.png)*']
['http://reddit.com/r/Jokes/comments/2e9hb9/another_russian_joke_i_love/)']
['https://www.youtube.com/watch?v=MZ35SOU9HTM']
['http://spikedmath.com/

['http://www.youtube.com/watch?nomobile=1&v=yIohIslPIMU']
['http://www.reddit.com/r/Jokes/comments/1b0sts/little_timmy_woke_up_one_morning_desperate_to_pee/).']
['http://www.house.gov/representatives/).']
['http://www.youtube.com/watch?v=r-iP5dh54T8']
['https://www.youtube.com/watch?v=q7P7BFBWQ1E)']
['http://assets0.ordienetworks.com/images/GifGuide/DealWithIt/tumblr_l6ouy08TEp1qz9muno1_r1_400.gif)']
['http://news.yahoo.com/physicists-found-higgs-boson-101311366.html)']
['http://en.wikipedia.org/wiki/Wat_%28food%29)']


In [50]:
df['joke'] = df['joke'].apply(lambda text: remove_urls(text))
df.head(1)

,score,joke
id,,
5tz52q,1,I hate how you cannot even say black paint any...


**Removal of numbers**  
* Here I delete only all the numbers that look like numbers, I will leave the written numbers for later.

In [51]:
df[text_col] = df[text_col].apply(lambda text: remove_numbers(text))
df.head(1)

,score,joke
id,,
5tz52q,1,I hate how you cannot even say black paint any...


In [52]:
print('The number of duplicated data is:',sum(df.duplicated()))

The number of duplicated data is: 213


## **Same jokes have different scores:**   
Now I take the biggest score from a joke and set this value to all duplicates, then I delete duplicates.

In [53]:
print('The number of duplicated data is:',sum(df['joke'].duplicated()))

The number of duplicated data is: 3764


In [54]:
dupls = df[df['joke'].duplicated() == True]['joke'].unique()
dupls

array(['What did the leper say to the prostitute? Keep the tip.',
       '!false It is funny because it is true.',
       "What is Mexico's National Animal? The drug mule.", ...,
       'What is the difference between ignorance and apathy? Do not know, do not care.',
       'How do you know if your roommate is gay? His dick tastes like shit.',
       'Why do they call it PMS? Because mad cow disease was already taken'],
      dtype=object)

In [55]:
dupls = df[df['joke'].duplicated() == True]['joke'].unique()

for j in dupls:
    max_score = max(set(df[df['joke'] == j]['score']))
    joke_idxs = df[df['joke'] == j].index
    df['score'][joke_idxs] = max_score
  

In [56]:
df[df['joke'] == dupls[0]]

,score,joke
id,,
5t6548,252,What did the leper say to the prostitute? Keep...
5swmk6,252,What did the leper say to the prostitute? Keep...
525zmr,252,What did the leper say to the prostitute? Keep...
4zzal9,252,What did the leper say to the prostitute? Keep...
4yf0xg,252,What did the leper say to the prostitute? Keep...
4tvbms,252,What did the leper say to the prostitute? Keep...
4t2ajn,252,What did the leper say to the prostitute? Keep...
48hd0j,252,What did the leper say to the prostitute? Keep...
46fpv4,252,What did the leper say to the prostitute? Keep...


Removal of duplicates

In [57]:
df.shape

(193906, 2)

In [58]:
df.drop_duplicates(inplace=True)
df.shape

(190142, 2)

In [59]:
# df.to_csv('/content/drive/My Drive/NLP_humor/data/jokes_dataset_PREPROC_withOrigScores.csv',
#             sep=',',
#             header=True, 
#             index=True,
#             encoding='utf-8')

### **Split 'score' in 4 quantiles (make 5 classes):**

In [59]:
df['score'].quantile([0.2, 0.4, 0.6, 0.8])

0.2     0.0
0.4     1.0
0.6     5.0
0.8    26.0
Name: score, dtype: float64

In [60]:
df.groupby(pd.cut(df['score'], np.percentile(df['score'], [20, 40, 60, 80]), include_lowest=True)).mean()

,score
score,
"(-0.001, 1.0]",0.239339
"(1.0, 5.0]",3.151722
"(5.0, 26.0]",12.500093


In [61]:
def add_rank(data):
      if data == 0:
        return 0
      elif (data > 0) and( data <= 1):
        return 1 
      elif (data > 1)and (data <= 5):
        return 2
      elif (data > 5)and (data <= 26):
        return 3
      elif data > 26:
        return 4

In [62]:
df['rank'] = df['score'].apply(add_rank)

In [63]:
df['rank'].value_counts()

0    60182
4    37757
3    37445
2    35822
1    18936
Name: rank, dtype: int64

**Drop joke from one word:**

In [64]:
df['text_word_count'] = df['joke'].apply(lambda x: len(str(x).split()))
df.head(1)

,score,joke,rank,text_word_count
id,,,,
5tz52q,1,I hate how you cannot even say black paint any...,1,22


In [65]:
df[df['text_word_count'] == 1]

,score,joke,rank,text_word_count
id,,,,
4nfz1o,0,Gandalf,0,1
4kq9zf,0,Bernie,0,1
441t69,0,Lol,0,1
3xunbs,0,-chan,0,1
3xo5ma,0,You.,0,1
...,...,...,...,...
2a6jyn,0,PSm,0,1
1p5dzb,0,Feminisim,0,1
1nju7e,941,Congress,4,1


In [66]:
df = df[df['text_word_count'] != 1]
df.drop(['score',	'text_word_count'], axis=1, inplace=True)

In [67]:
df.head()

,joke,rank
id,,
5tz52q,I hate how you cannot even say black paint any...,1
5tz4dd,What is the difference between a Jew in Nazi G...,0
5tz319,I recently went to America.... ...and being th...,0
5tz2wj,"Brian raises his hand and says, ""He is in Heav...",1
5tz1pc,You hear about the University book store worke...,0


In [68]:
df.shape

(190056, 2)

In [105]:
df.isna().sum()

joke    0
rank    0
dtype: int64

# **Split**

Train/Test/Val

In [69]:
data_train, data_test = train_test_split(df, test_size=0.1, random_state=42)

data_train, data_val = train_test_split(data_train, test_size=0.1, random_state=42)

data_train.index = range(len(data_train))
data_val.index = range(len(data_val))

print("Train size = ", len(data_train))
print("Validation size = ", len(data_val))
print("Test size = ", len(data_test))

Train size =  153945
Validation size =  17105
Test size =  19006


# **Augmentation**

Just for train and val datasets. Test dataset is keeped clean.

### **nlpaug**

## **SynonymAug**

In [70]:
!pip install nlpaug

Defaulting to user installation because normal site-packages is not writeable


You should consider upgrading via the 'C:\Users\anaconda3\python.exe -m pip install --upgrade pip' command.


In [71]:
import nlpaug.augmenter.char as nac
import nlpaug.augmenter.sentence as nas
import nlpaug.augmenter.word as naw
import nlpaug.augmenter.word.context_word_embs as nawcwe
import nlpaug.augmenter.word.word_embs as nawwe
import nlpaug.augmenter.word.spelling as naws

In [77]:
txt = 'I run at school.'

In [82]:
aug_syn = naw.SynonymAug(aug_src='wordnet', aug_min=1, aug_max=1)

In [83]:
def set_aug(txt):
    return aug_syn.augment(txt)

In [84]:
%%time
train_DF = data_train.copy(deep=True)

train_DF['joke_aug'] = train_DF['joke'].apply(set_aug)

Wall time: 10min 58s


In [85]:
train_DF.drop(['joke'], axis=1, inplace=True)

In [86]:
data_train.shape

(153945, 2)

In [87]:
train_DF.shape

(153945, 2)

In [88]:
train_DF.rename({'joke_aug': 'joke'}, axis=1,  inplace=True) 

In [89]:
new_df = data_train.append(train_DF, ignore_index=True, sort=False)

In [90]:
new_df

,joke,rank
0,Did you hear about the lumberjack who lost his...,3
1,What is the difference between a girl in a chu...,3
2,"If the black man got shot, these guys get fire...",0
3,What do you call it when your lizard does not ...,2
4,"What do you call , Muslims who meet to protest...",0
...,...,...
307885,1 seen twins hugging I told them to get a womb,0
307886,When someone replies late. .. If NASA can find...,2
307887,There is a giant hole in town and everybody is...,4
307888,I get so angry when I see nudity on Reddit. .....,0


In [91]:
new_df.isna().sum()

joke    0
rank    0
dtype: int64

### Shuffle

In [92]:
shuf_df = new_df.reindex(np.random.permutation(new_df.index)).reset_index(drop=True)

In [93]:
shuf_df

,joke,rank
0,Why is ISIS going after programs? Because they...,2
1,How are women and hurricanes alike? The both m...,3
2,"What is an ""innuendo"" ? An Italian suppository",3
3,What do you call a night watchman with deep-se...,3
4,My girlfriend had a sexual fantasy to roleplay...,3
...,...,...
307885,What do you call bacteria that can swim fast? ...,3
307886,"When my SO is sad, she likes to code. So I giv...",3
307887,Q - How many billionaires does it take to make...,3
307888,"Raging bull SEX POSITION Tried a new position,...",0


### Split train/val

In [102]:
# data_train, data_val = shuf_df[shuf_df["indic"].eq("train")], shuf_df[shuf_df["indic"].eq("val")]

In [104]:
# data_train['indic'].unique()

array(['train'], dtype=object)

In [105]:
# data_val['indic'].unique()

array(['val'], dtype=object)

In [108]:
# data_val.drop(['indic'], axis=1, inplace=True)
# data_train.drop(['indic'], axis=1, inplace=True)

In [94]:
data_val.reset_index(drop=True, inplace=True)
data_val.head()

,joke,rank
0,Give a man a match and he will be warm for a f...,3
1,Why did Leonardo DiCaprio laugh at the Oscar j...,0
2,"So, tell me. Why do Russians go crazy over pho...",1
3,Why do people buy smart cars? Because opposite...,2
4,What does a South Carolina cop do when a panda...,4


In [95]:
data_test.reset_index(drop=True, inplace=True)
data_test.head()

,joke,rank
0,what do you call a tilt-a-whirl operator with ...,2
1,The three tortures. A tourist got lost hiking ...,4
2,"A Black guy, an Indian guy, and Donald Trump w...",2
3,I do not get this joke A guy woke up after bei...,2
4,Yo mama Yo mama's got so many balls lined up r...,0


# **Save preproc data:**

In [96]:
shuf_df.to_csv(r'C:\NLP_humor\data\jokes_TRAIN.csv',
            sep=',',
            header=True, 
            index=False,
            encoding='utf-8')

In [97]:
data_val.to_csv(r'C:\NLP_humor\data\jokes_VAL.csv',
            sep=',',
            header=True, 
            index=False,
            encoding='utf-8')

In [98]:
data_test.to_csv(r'C:\NLP_humor\data\jokes_TEST.csv',
            sep=',',
            header=True, 
            index=False,
            encoding='utf-8')

### Load & Read data: 

In [99]:
train = pd.read_csv(r'C:\NLP_humor\data\jokes_TRAIN.csv', 
                 sep=',', 
                 encoding='utf-8')

In [100]:
train.head()

,joke,rank
0,Why is ISIS going after programs? Because they...,2
1,How are women and hurricanes alike? The both m...,3
2,"What is an ""innuendo"" ? An Italian suppository",3
3,What do you call a night watchman with deep-se...,3
4,My girlfriend had a sexual fantasy to roleplay...,3


In [101]:
train.isna().sum()

joke    3
rank    0
dtype: int64

In [102]:
train.dropna(inplace=True)